# 0 Imports

In [1]:
import pickle
import datetime
import inflection

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot

from IPython.display        import Image
from IPython.core.display   import HTML

In [2]:
# Supressão da notação científica.
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.6f}'.format)

## 0.1 Funções Suporte

In [3]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [12,6]
    plt.rcParams['font.size'] = 20
    
    display( HTML('<style>.container { width: 100% !important;} </style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
def abrir_arquivo()-> pd.DataFrame:
    with open(f"{__CAMINHO_INTERIM}{__NOME_ARQUIVO_IMPORTAR}","rb") as arquivo:
        df = pickle.load(arquivo)
    return df

In [5]:
def visualizacao_dados_categoricos(df):
    for i in df:
        print(f'Atributo: {i}') 
        print(f'Valores Únicos: {len(df[i].sort_values().unique())}')
        print(f'Valores Descritos: {df[i].sort_values().unique().tolist()}\n')

In [6]:
def ajustes_id(df: pd.DataFrame)-> pd.DataFrame:
    df['subscription_guid'] = df['subscription_guid'].str.strip().str.replace('-', '_', regex=False)
    df['resource_guid'] = df['resource_guid'].str.strip().str.replace('-', '_', regex=False)
    return df

In [7]:
def ajuste_ordenamento(df: pd.DataFrame)-> pd.DataFrame:
    df = df.sort_values(by=['maquina', 'date']).reset_index() #Garantir a sequencia ordenada de máquina e data
    df.drop(columns=['index','subscription_name'], inplace=True)
    return df

In [8]:
def fe_date(df:pd.DataFrame)-> pd.DataFrame:
    # year
    df['ano'] = df['date'].dt.year
    # month
    df['mes'] = df['date'].dt.month
    # day
    df['dia'] = df['date'].dt.day
    # week of year
    df['semana_ano'] = df['date'].dt.isocalendar().week
    # year week string
    df['ano_semana'] = df['date'].dt.strftime('%Y-%W')
    # year month
    df['ano_mes'] = df['date'].dt.strftime('%Y-%m')
    
    df['dia_da_semana'] = df['date'].dt.dayofweek.astype(int)
    
    df['fim_de_semana'] = df['dia_da_semana'].apply(lambda x : 1 if x >= 5 else 0)
    
    return df

In [9]:
def fe_cost_fixo(df:pd.DataFrame)-> pd.DataFrame:
    
    df['custo_fixo_diario'] = df.groupby(['maquina','date'])['cost'].transform('sum')
    
    df['custo_fixo_semanal'] = df.groupby(['maquina','ano_semana'])['cost'].transform('sum')
    
    df['custo_fixo_mensal'] = df.groupby(['maquina','ano_mes'])['cost'].transform('sum')
    
    return df

In [10]:
def fe_cost_movel(df:pd.DataFrame)-> pd.DataFrame:
    
    custo_diario = df.groupby(["date", "maquina"])["cost"].sum().reset_index()
    
    custo_diario["custo_soma_movel_semanal"] = custo_diario.groupby("maquina")["cost"].transform(lambda x: x.rolling(window=7, min_periods=1).sum())
    
    custo_diario["custo_soma_movel_mensal"] = custo_diario.groupby("maquina")["cost"].transform(lambda x: x.rolling(window=30, min_periods=1).sum())
    
    custo_diario.drop(columns='cost', inplace=True)
    
    return pd.merge(df, custo_diario, how='left', on=['date','maquina'])

In [11]:
def exportar_df(df: pd.DataFrame):
    df.to_pickle(f'{__CAMINHO_INTERIM}{__NOME_ARQUIVO_EXPORTAR}')

## 0.2 Load Data

In [12]:
__CAMINHO_RAW = '../data/raw/'
__CAMINHO_INTERIM = '../data/interim/'

__NOME_ARQUIVO_IMPORTAR = '1_0_descricao.pkl'
__NOME_ARQUIVO_EXPORTAR = '3_0_feature_engineering.pkl'

In [13]:
df = abrir_arquivo()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59331 entries, 0 to 59330
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subscription_name  59331 non-null  object        
 1   subscription_guid  59331 non-null  object        
 2   date               59331 non-null  datetime64[ns]
 3   resource_guid      59331 non-null  object        
 4   service_name       59331 non-null  object        
 5   service_type       59331 non-null  object        
 6   service_region     59331 non-null  object        
 7   service_resource   59331 non-null  object        
 8   quantity           59331 non-null  float64       
 9   cost               59331 non-null  float64       
 10  maquina            59331 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 5.0+ MB


In [14]:
df.iloc[len(df):len(df)-500:-50]

,subscription_name,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina
59330,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-31,40551b4c-e8be-48ed-b70b-f8d25c7de724,Storage,General Block Blob,All,Read Operations,0.000800,0.000000,unicin
59280,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-21,2dba9884-2518-4427-90ff-d2c100c5f775,Storage,Standard SSD Managed Disks,BR South,E6 LRS Disk,0.032256,0.289008,unicin
59230,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-31,9c150bf9-2bad-430e-a53c-c213804f49ef,Virtual Network,IP Addresses,All,Standard IPv4 Static Public IP,26.000000,0.130000,unicin
59180,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-21,43034668-a244-4031-bb4d-be5bcce22c5f,Bandwidth,Inter-Region,Intercontinental,Inter Continent Data Transfer Out - LATAM To Any,0.000134,0.000009,unicin
59130,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-31,cd4cfb51-f027-4d88-89d0-80e78795f00e,Backup,All,BR South,GRS Data Stored,0.806124,0.049176,unicin
59080,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-21,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,Storage,Tables,All,Batch Write Operations,0.029200,0.000000,unicin
59030,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-31,c089a13a-9dd0-44b5-aa9e-44a77bbd6788,Bandwidth,All,BR South,Standard Data Transfer Out,0.351183,0.063564,unicin
58980,Microsoft Azure Sponsorship,15dc64f3-696a-48fc-9169-8467e3f7bba0,2024-10-21,6db4a779-0d75-481a-af26-026ccf5795c8,Bandwidth,Inter-Region,South America,Intra Continent Data Transfer Out,0.000000,0.000000,unicin
58930,Microsoft Azure Sponsorship,82375e17-0dda-4790-b6e8-c565d4f08e87,2024-10-31,0b608a26-f611-4232-8192-ce81b6b57194,Azure DNS,All,All,Private Zone,0.017472,0.008736,portal
58880,Microsoft Azure Sponsorship,82375e17-0dda-4790-b6e8-c565d4f08e87,2024-10-21,43034668-a244-4031-bb4d-be5bcce22c5f,Bandwidth,Inter-Region,Intercontinental,Inter Continent Data Transfer Out - LATAM To Any,1.648730,0.263793,portal


In [15]:
df = ajustes_id(df)
df = ajuste_ordenamento(df)
df = fe_date(df)
df = fe_cost_fixo(df)
df = fe_cost_movel(df)


In [16]:
df.iloc[len(df):len(df)-500:-50]

,subscription_guid,date,resource_guid,service_name,service_type,service_region,service_resource,quantity,cost,maquina,ano,mes,dia,semana_ano,ano_semana,ano_mes,dia_da_semana,fim_de_semana,custo_fixo_diario,custo_fixo_semanal,custo_fixo_mensal,custo_soma_movel_semanal,custo_soma_movel_mensal
59330,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-31,40551b4c_e8be_48ed_b70b_f8d25c7de724,Storage,General Block Blob,All,Read Operations,0.000800,0.000000,unicin,2024,10,31,44,2024-44,2024-10,3,0,2.758325,18.769080,170.831834,34.837048,165.533815
59280,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-29,b9e5e77c_a0b3_4a2c_9b8b_57fa54f31c52,Storage,Tables,All,Batch Write Operations,0.029200,0.000000,unicin,2024,10,29,44,2024-44,2024-10,1,0,5.365564,18.769080,170.831834,37.503686,168.086228
59230,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-26,9c150bf9_2bad_430e_a53c_c213804f49ef,Virtual Network,IP Addresses,All,Standard IPv4 Static Public IP,48.000000,0.240000,unicin,2024,10,26,43,2024-43,2024-10,5,1,5.464043,37.642259,170.831834,37.705279,168.274790
59180,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-24,6db4a779_0d75_481a_af26_026ccf5795c8,Bandwidth,Inter-Region,South America,Intra Continent Data Transfer Out,0.000000,0.000000,unicin,2024,10,24,43,2024-43,2024-10,3,0,5.412130,37.642259,170.831834,37.891774,168.312476
59130,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-21,cd4cfb51_f027_4d88_89d0_80e78795f00e,Backup,All,BR South,GRS Data Stored,1.612248,0.098352,unicin,2024,10,21,43,2024-43,2024-10,0,0,5.410196,37.642259,170.831834,39.227063,169.191592
59080,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-18,2f2a1523_d8af_5cea_9b91_d0285e17e343,Backup,All,BR South,Azure VM Protected Instances,0.016128,0.161280,unicin,2024,10,18,42,2024-42,2024-10,4,0,5.444263,39.423166,170.831834,39.420624,169.144926
59030,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-16,c089a13a_9dd0_44b5_aa9e_44a77bbd6788,Bandwidth,All,BR South,Standard Data Transfer Out,2.141641,0.387638,unicin,2024,10,16,42,2024-42,2024-10,2,0,5.494795,39.423166,170.831834,38.421657,169.068528
58980,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-13,f1fcf7da_9502_4a86_bfe7_9c2646efa203,Storage,Standard SSD Managed Disks,BR South,E4 LRS Disk,0.096768,0.433512,unicin,2024,10,13,41,2024-41,2024-10,6,1,5.403827,37.349604,170.831834,37.349604,167.654203
58930,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-10,8dbc6497_3d3a_47d6_9de4_dd42072fad66,Storage,Standard Page Blob,BR South,LRS Data Stored,0.001248,0.000144,unicin,2024,10,10,41,2024-41,2024-10,3,0,5.629300,37.349604,170.831834,38.150458,168.906321
58880,15dc64f3_696a_48fc_9169_8467e3f7bba0,2024-10-08,26ce34b7_67b3_480d_9d1b_54a7fb80f67a,Virtual Network,IP Addresses,All,Basic IPv4 Static Public IP,24.000000,0.086400,unicin,2024,10,8,41,2024-41,2024-10,1,0,5.600468,37.349604,170.831834,43.315850,168.616437


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59331 entries, 0 to 59330
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   subscription_guid         59331 non-null  object        
 1   date                      59331 non-null  datetime64[ns]
 2   resource_guid             59331 non-null  object        
 3   service_name              59331 non-null  object        
 4   service_type              59331 non-null  object        
 5   service_region            59331 non-null  object        
 6   service_resource          59331 non-null  object        
 7   quantity                  59331 non-null  float64       
 8   cost                      59331 non-null  float64       
 9   maquina                   59331 non-null  object        
 10  ano                       59331 non-null  int32         
 11  mes                       59331 non-null  int32         
 12  dia               

In [18]:
#colunas = ['service_name', 'service_type', 'service_region', 'service_resource','maquina']
#colunas_categoricas = df.loc[:,colunas]
#visualizacao_dados_categoricos(colunas_categoricas)

## 0.3 Planejamento para criação de novas Features

0b1856f5_ffb1_4fb2_9b31_ebc3ddeacf68 - CEAEC

15dc64f3_696a_48fc_9169_8467e3f7bba0 - UNICIN

82375e17_0dda_4790_b6e8_c565d4f08e87 - PORTAL

Observar (prox iteração) com mais cuidado as colunas:
subscription_guid; resource_guid


1. É Semana
2. É fim de semana
3. Custo Diário
4. Média Móvel Semanal
5. Média Móvel Mensal

Contagem de uso do serviço.
Recursos baseados em região.
Recursos baseados em tempo.


##### Time-based features
df_features['day_of_week'] = df_features['date'].dt.dayofweek
df_features['is_weekend'] = df_features['date'].dt.dayofweek.isin([5,6]).astype(int)

##### Cost-based features
df_features['daily_avg_cost'] = df_features.groupby(['maquina', 'date'])['cost'].transform('mean')
df_features['cost_rolling_mean'] = df_features.groupby('maquina')['cost'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df_features['cost_rolling_std'] = df_features.groupby('maquina')['cost'].transform(lambda x: x.rolling(window=7, min_periods=1).std())

##### Service-based features
df_features['service_daily_count'] = df_features.groupby(['service_name', 'date'])['cost'].transform('count')
df_features['service_type_daily_cost'] = df_features.groupby(['service_type', 'date'])['cost'].transform('sum')

##### Region-based features
df_features['region_daily_cost'] = df_features.groupby(['service_region', 'date'])['cost'].transform('sum')

# 3.0 F.E.

## 3.1 Próximas implementações

In [19]:
df_fe = df[['maquina','date','cost','custo_fixo_diario']].copy()

In [20]:
# Não deu certo desse jeito, tive que fazer do outro, não sei o pq, ainda...
# df_fe['custo_media_movel_semanal'] = df_fe.loc[:,['maquina','date','cost']].groupby(
#                                     ['maquina']
#                                     )['cost'].transform(
#                                                 lambda x: x.rolling(
#                                                     window=7,
#                                                     min_periods=1
#                                                     ).sum()
#                                                 )

In [21]:
# Não deu certo desse jeito, tive que fazer do outro, não sei o pq, ainda...
# df_fe['custo_media_movel_semanal_teste'] = df_fe.groupby(
#                                                     ['maquina']
#                                                     )['cost'].transform(
#                                                                 lambda x: x.rolling(
#                                                                     window=7,
#                                                                     min_periods=1
#                                                                     ).sum()
#                                                                 ).reset_index(
#                                                                     level=0,
#                                                                     drop=True
#                                                                     )

In [22]:
# Não deu certo desse jeito, tive que fazer do outro, não sei o pq, ainda...
# df_fe['custo_media_movel_semanal_data'] = df_fe.groupby(
#                                             ['maquina',
#                                             'date'])['cost'].transform(
#                                                         lambda x: x.rolling(
#                                                             window=7,
#                                                             min_periods=1
#                                                             ).sum()
#                                                         )

In [23]:
# Não deu certo desse jeito, tive que fazer do outro, não sei o pq, ainda...
# df_fe['custo_media_movel_mensal'] = df_fe.groupby(
#                                     ['maquina']
#                                     )['cost'].transform(
#                                                 lambda x: x.rolling(
#                                                     window=30,
#                                                     min_periods=1
#                                                     ).sum()
#                                                 )

In [24]:
# aux2 = df_fe.loc[(df_fe['date']>='2024-09-12') & (df_fe['maquina'] == 'ceaec'),:].groupby(['maquina','date'])['cost'].sum().reset_index()

In [25]:
# aux2['cost'].sum()

In [ ]:
#PRECISO FAZER UM MERGE AQUI ENTRE df_fe E df2
# df_fe.info()

In [ ]:
# aux= df2.loc[(df2['maquina']=='unicin') & (df2['date']=='2024-10-11'),
#         ['date',
#         'maquina',
#         'cost',
#         'custo_diario',
#         'custo_media_movel_semanal',
#         'custo_media_movel_semanal_teste']]
# aux

In [ ]:
# aux['cost'].sum()

In [ ]:
# aux_df = df2.loc[(df2['maquina']=='unicin') & (df2['date']>='2024-09-12'),
#         ['date',
#         'maquina',
#         'cost',
#         'custo_media_movel_semanal',
#         'custo_media_movel_mensal']
#         ].groupby(['maquina','date']).sum().sort_values(by='date',ascending=False).reset_index()
# aux_df

In [ ]:
# aux_df.loc[:,'cost'].sum()

In [ ]:
# aux_df.loc[aux_df['date']>='2024-10-05',['cost']].sum()

# 3.9 Export PKL

In [ ]:
exportar_df(df=df)